In [ ]:
import os
import json
import time
from airops.defaults import make_chain, make_agent
from airops import models, prompts
from airops.tools import get_action_details, tavily_search, tavily_extract
from pydantic import BaseModel, Field
from typing import Dict, List, Any
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import RateLimitError
from langfuse.decorators import observe, langfuse_context
from langfuse.callback import CallbackHandler
from tqdm import tqdm
import joblib


with open('./context/workflow_spec.md', 'r') as f:
    WORKFLOW_SPEC = f.read()

with open('./context/integration_actions.json', 'r') as f:
    INTEGRATION_ACTIONS = sorted(json.load(f), key = lambda x: x['integration'])
    for action in INTEGRATION_ACTIONS:
        action["inputs_schema"] = [field for field in action["inputs_schema"] if field["interface"] != "integration"]

AVAILABLE_INTEGRATION_ACTIONS = [{'integration': ia['integration'], 'action': ia['action']} for ia in INTEGRATION_ACTIONS]

SAMPLE_WORKFLOW_CONTEXTS = []
for file in os.listdir('./context/sample_workflows/'):
    fp = os.path.join('./context/sample_workflows/', file)
    with open(fp, 'r') as file:
        SAMPLE_WORKFLOW_CONTEXTS.append(json.load(file))

In [ ]:
@observe()
def create_test_case(integration_action, sample_context):
    try:
        create_test_case_agent = make_agent(
            tools = [get_action_details, tavily_search, tavily_extract],
            prompt_template = prompts.CREATE_TEST_CASE_PROMPT,
            output_model = models.TestCase,
            llm='gpt-4.1-mini'
        )
        langfuse_handler = langfuse_context.get_current_langchain_handler()
        results = create_test_case_agent.invoke({
            "workflow_context": sample_context,
            "integration": integration_action["integration"],
            "action": integration_action["action"],
        }, config={"callbacks": [langfuse_handler]})
        return {
            **integration_action,
            'context': sample_context,
            **results.model_dump()
        }
    except RateLimitError as e:
        time.sleep(60)
        return create_test_case(integration_action, sample_context)
    except Exception as e:
        print(f'Error: {e}')
        return None

In [ ]:
def create_test_cases(target_fp):
    if os.path.exists(target_fp):
        return joblib.load(target_fp)
    test_cases = []
    for action in tqdm(AVAILABLE_INTEGRATION_ACTIONS, desc="Actions"):
        futures = []
        with ThreadPoolExecutor() as executor:
            for sample in SAMPLE_WORKFLOW_CONTEXTS:
                futures.append(executor.submit(create_test_case, action, sample))
            for f in as_completed(futures):
                result = f.result()
                if result is not None:
                    test_cases.append(result)
    joblib.dump(test_cases, target_fp)
    return test_cases

In [ ]:
test_cases = create_test_cases('./eval/test_cases.joblib')

In [ ]:
test_cases[1]